In [1]:
import torch
from torch import nn
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset, Dataset
from audio_dataset import DistributedTorchDataset, AudioDataLoader  
from typing import List
from trainer import AudioTrainer
import os
import pandas as pd

from unet_dac import UnetDAC

In [2]:
from config import NUM_MICS, ANGLE_RES


L = 96
K = 256
INPUT_LEN = 64
VIRTUAL_BATCH_SIZE = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UnetDAC(L=L, K=K, M=NUM_MICS).to(device)

In [3]:
# inputs = torch.Tensor(np.random.random((INPUT_LEN, 2*(NUM_MICS - 1), K, L))).to(device)
# outputs = torch.Tensor(np.random.randint((INPUT_LEN, NUM_MICS))).to(device)
# print(f'Input shape: {inputs.shape}')
# res = model(inputs)
# print(f"res shape: {res.shape}")

In [4]:
# dataset = DistributedTorchDataset('data_batches', 'train' , ['samples', 'ref_stft', 'target'])
# dataset = DistributedTorchDataset('data_batches', 'train' , ['samples', 'ref_stft', 'target'])
# criterion = nn.CrossEntropyLoss()
# lr: float = 0.001
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# trainer = AudioTrainer(model=model, loss_fn=criterion, optimizer=optimizer)
# trainer.fit(dataset, dataset, num_epochs=100, checkpoints='checkpoints', early_stopping=3, print_every=10)



In [6]:
NUM_TRAIN_EPOCHS = 100
lr: float = 2e-3
epochs: int = 100
early_stopping: int = 3
mininbatch_size: int = 16

train_statistics = []
dataset = DistributedTorchDataset('data_batches', 'train' , ['samples', 'ref_stft', 'target'], virtual_batch_size=1)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


ValueError: too many values to unpack (expected 2)

### Train loop - manual ###

In [ ]:
losses = []
for epoch in range(epochs):
    running_loss = 0.0
    print(f'--- epoch {epoch + 1} ---')

    # Train on minibatches
    for i, minibatch in tqdm(enumerate(dataloader), desc=f'Epoch {epoch} trainig batches', total=len(dataloader)):
        samples, ref_abs_square, target = minibatch
        # print(f"hello I am a minibatch! my dimensions are:")
        # print(f"samples.shape={samples.shape}\nref_stft.shape={ref_stft.shape}\ntarget.shape={target.shape}")
        # Forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(samples)
        # TODO
        # output_directions = torch.dot(outputs, ref_stft * ref_stft.T)
        # output_angle = torch.argmax(output_directions, axis=1)
        loss = criterion(outputs, target // ANGLE_RES)
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() / VIRTUAL_BATCH_SIZE
        losses.append({'epoch': epoch, 'batch': i, 'loss': loss.item()})
        if i % 8 == 7:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss :.3f}')
            # running_loss = 0.0

    # Validation on minibatches

--- epoch 1 ---


Epoch 0 trainig batches:   0%|          | 0/64 [00:00<?, ?it/s]

Epoch 0 trainig batches:   5%|▍         | 3/64 [03:28<58:53, 57.92s/it]   

In [ ]:
loss_df = pd.DataFrame(losses)
display(loss_df)
print(f"Average loss: {loss_df.loss.mean()}")

,epoch,batch,loss
0,0,0,2.178989
1,0,1,1.925689
2,0,2,1.894877
3,0,3,1.807512
4,0,4,2.468046
5,0,5,2.434453
6,0,6,2.485450
7,0,7,2.507769
8,0,8,2.415939
9,0,9,2.450812


Average loss: 2.3701120615005493


In [ ]:
from transformers import Trainer, TrainingArguments
import torch.nn as nn
import torch.optim as optim

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=100,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    learning_rate=0.001,             # learning rate
)

optimizer = optim.Adam(model.parameters(), lr=training_args.learning_rate)
loss_fn = nn.CrossEntropyLoss()

# Define the Hugging Face Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset,            # training dataset
    eval_dataset=None,                   # evaluation dataset
    compute_metrics=None,                # any additional metrics you want to compute
    optimizers=(optimizer, None),        # (optimizer, scheduler), scheduler is None here
    data_collator=None                   # data collator, default collate_fn for torch DataLoader
)

trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: agadish (the-muppets). Use `wandb login --relogin` to force relogin


  0%|          | 0/6400 [00:00<?, ?it/s]

TypeError: vars() argument must have __dict__ attribute